In [1]:
# Imports and set torch device
import torch
import meshplot as mp
import kaolin as kal
import matplotlib.pyplot as plt
import trimesh
import numpy as np
import open3d as o3d
import open3d.core as o3c

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'bookshelf'
obj_path = 'data/demo/' + object + '.obj'

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Torch will run on: cuda:0


In [2]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices = mesh.vertices.cpu().numpy()
faces = mesh.faces.cpu().numpy()
colors = mesh.vertex_normals.cpu().numpy()
vertices = (vertices - np.mean(vertices, axis=0)) / np.std(vertices)
# Sample Point Cloud
np.random.seed(42)
n_samples = int(vertices.shape[0] * 2.)
trimeshMesh = trimesh.Trimesh(vertices, faces)
point_cloud = np.asarray(trimesh.sample.sample_surface_even(trimeshMesh, n_samples)[0])
# point_cloud = (point_cloud - np.mean(point_cloud, axis=0)) / np.std(point_cloud)
print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])
print('Number of sampled points:', point_cloud.shape[0])
mp.plot(point_cloud, shading={'point_size':0.2})
plt.show()

Number of vertices:  5786
Number of faces:  8624
Number of sampled points: 11572


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.003204…

## Using Alpha Shapes

In [3]:
alpha = 0.15
o3d_points = o3d.cpu.pybind.utility.Vector3dVector(point_cloud)
pcd = o3d.cpu.pybind.geometry.PointCloud(o3d_points)
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
remeshed_vertices = np.asarray(mesh.vertices)
remeshed_faces = np.asarray(mesh.triangles)
print('Number of remeshed vertices: ', remeshed_vertices.shape[0])
print('Number of remeshed faces: ', remeshed_faces.shape[0])
mp.plot(remeshed_vertices, remeshed_faces, np.random.rand(remeshed_faces.shape[0]))
plt.show()

Number of remeshed vertices:  8759
Number of remeshed faces:  17514


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.003204…

## Poisson Surface Reconstruction

In [4]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    pcd.estimate_normals()
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
densities = np.asarray(densities)
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)
remeshed_vertices = np.asarray(mesh.vertices)
remeshed_faces = np.asarray(mesh.triangles)
print('Number of remeshed vertices: ', remeshed_vertices.shape[0])
print('Number of remeshed faces: ', remeshed_faces.shape[0])
mp.plot(remeshed_vertices, remeshed_faces, np.random.rand(remeshed_faces.shape[0]))
# mp.plot(remeshed_vertices, remeshed_faces, np.asarray(densities))
plt.show()

[Open3D DEBUG] Input Points / Samples: 11572 / 11572
[Open3D DEBUG] #   Got kernel density: 0.0380001 (s), 417.211 (MB) / 417.211 (MB) / 417 (MB)
[Open3D DEBUG] #     Got normal field: 0.023 (s), 421.758 (MB) / 421.758 (MB) / 421 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.290125e-04 / 3.807333e+00
[Open3D DEBUG] #       Finalized tree: 0.0420001 (s), 433.258 (MB) / 433.258 (MB) / 433 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.0319998 (s), 427.375 (MB) / 433.258 (MB) / 433 (MB)
[Open3D DEBUG] #Set point constraints: 0.0110002 (s), 427.414 (MB) / 433.258 (MB) / 433 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 433518 / 124896 / 370553
[Open3D DEBUG] Memory Usage: 427.414 MB
[Open3D DEBUG] # Linear system solved: 0.124 (s), 431.676 (MB) / 433.258 (MB) / 433 (MB)
[Open3D DEBUG] Got average: 0.00299978 (s), 431.719 (MB) / 433.258 (MB) / 433 (MB)
[Open3D DEBUG] Iso-Value: 5.010170e-01 = 5.797769e+03 / 1.157200e+04
[Open3D DEBUG] #          Total Solve:       0.6 (

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1734335…